# Additional analysis with R

In this notebook, we analyze the biome data when the resolution of the hex grid is higher than in the main text.

In [1]:
library(plotbiomes)
library(ggplot2)
library(raster)
library(maptools)
library(sp)
library(EMT)
data_null=read.csv("Null_Biome_distributions2.csv") # null data corresponding all terestorial ares


     Happy biome plotting!

 要求されたパッケージ sp をロード中です 

Checking rgeos availability: TRUE
Please note that 'maptools' will be retired during 2023,
plan transition at your earliest convenience;
some functionality will be moved to 'sp'.



In [2]:
data_null

X,hex_index2,Species,Annu_Mean_Temp,Annu_Prec
<int>,<chr>,<chr>,<dbl>,<dbl>
0,82a847fffffffff,anteater,25.055918,1032.000
1,82b34ffffffffff,anteater,22.494013,583.000
3,825f2ffffffffff,anteater,26.421509,1672.000
4,82a85ffffffffff,anteater,25.209452,1159.000
7,826657fffffffff,anteater,26.627391,2580.000
10,82a81ffffffffff,anteater,21.577311,1047.000
12,82660ffffffffff,anteater,21.504501,1528.000
44,826627fffffffff,anteater,26.228910,1139.933
45,828adffffffffff,anteater,26.925288,1861.000


In [3]:
# classification on Whitker biome 
Biome_class=function(data){
    df1=Whittaker_biomes[Whittaker_biomes$biome_id==1, ]
    pol1.x=df1$temp_c
    pol1.y=df1$precp_cm
    
    df2=Whittaker_biomes[Whittaker_biomes$biome_id==2, ]
    pol2.x=df2$temp_c
    pol2.y=df2$precp_cm
    
    df3=Whittaker_biomes[Whittaker_biomes$biome_id==3, ]
    pol3.x=df3$temp_c
    pol3.y=df3$precp_cm
    
    df4=Whittaker_biomes[Whittaker_biomes$biome_id==4, ]
    pol4.x=df4$temp_c
    pol4.y=df4$precp_cm
    
    df5=Whittaker_biomes[Whittaker_biomes$biome_id==5, ]
    pol5.x=df5$temp_c
    pol5.y=df5$precp_cm
    
    df6=Whittaker_biomes[Whittaker_biomes$biome_id==6, ]
    pol6.x=df6$temp_c
    pol6.y=df6$precp_cm
    
    df7=Whittaker_biomes[Whittaker_biomes$biome_id==7, ]
    pol7.x=df7$temp_c
    pol7.y=df7$precp_cm
    
    df8=Whittaker_biomes[Whittaker_biomes$biome_id==8, ]
    pol8.x=df8$temp_c
    pol8.y=df8$precp_cm
    
    df9=Whittaker_biomes[Whittaker_biomes$biome_id==9, ]
    pol9.x=df9$temp_c
    pol9.y=df9$precp_cm
    
    category=c()
    X=data$Annu_Mean_Temp
    Y=data$Annu_Prec 
    for (i in 1:nrow(data)){
        X=data$Annu_Mean_Temp[i]
        Y=data$Annu_Prec[i]/10 # convert mm to cm
        if (point.in.polygon(X, Y, pol1.x, pol1.y)==1){
            category=c(category, 1)
        }
        else 
            if (point.in.polygon(X, Y, pol2.x, pol2.y)==1){
            category=c(category, 2)
        }
        else if (point.in.polygon(X, Y, pol3.x, pol3.y)==1){
            category=c(category, 3)
        }
        else if (point.in.polygon(X, Y, pol4.x, pol4.y)==1){
            category=c(category, 4)
        }
        else if (point.in.polygon(X, Y, pol5.x, pol5.y)==1){
            category=c(category, 5)
        }
        else if (point.in.polygon(X, Y, pol6.x, pol6.y)==1){
            category=c(category, 6)
        }
        else if (point.in.polygon(X, Y, pol7.x, pol7.y)==1){
            category=c(category, 7)
        }
        else if (point.in.polygon(X, Y, pol8.x, pol8.y)==1){
            category=c(category, 8)
        }
        else if (point.in.polygon(X, Y, pol9.x, pol9.y)==1){
            category=c(category, 9)
        }
        else{
        
            category=c(category, 10)
        }
        
    }
    #print(category)
    data$biome_id=as.factor(category)
    return(data)
}
data=Biome_class(data_null)

In [4]:
# obtain the frequencies of the terestrial
p_null=c(nrow(data[data$biome_id==1,]),nrow(data[data$biome_id==2,]),
        nrow(data[data$biome_id==3,]), nrow(data[data$biome_id==4,]),
           nrow(data[data$biome_id==5,]), nrow(data[data$biome_id==6,]),
           nrow(data[data$biome_id==7,]),nrow(data[data$biome_id==8,]),
         nrow(data[data$biome_id==9,]),nrow(data[data$biome_id==10,]))/nrow(data) # we skip id==3 as it does not exist
p_null # null distribution of biome

[1] 0.239753738 0.162532982 0.002990325 0.039988273 0.175432425 0.034066256
 [7] 0.084315450 0.087950748 0.123365582 0.049604222

In [6]:
#obtrain biome of real animals
data_RA=read.csv("Real_Biome_distributions2.csv") # THIS  FILE MAY LACK SOME DATA. NEED TO CHECK TOMORROW
head(data_RA)

,X,hex_index2,Species,Annu_Mean_Temp,Annu_Prec
,<int>,<chr>,<chr>,<dbl>,<dbl>
1,0,82a847fffffffff,anteater,25.05592,1032
2,1,82b34ffffffffff,anteater,22.49401,583
3,3,825f2ffffffffff,anteater,26.42151,1672
4,4,82a85ffffffffff,anteater,25.20945,1159
5,7,826657fffffffff,anteater,26.62739,2580
6,10,82a81ffffffffff,anteater,21.57731,1047


In [7]:
# TEST 1 compare distirbution of real animals vs null distributions
# is the distributions of RAs differ from null
pvals=c()
species=unique(data_RA$Species)
for (i in 1:length(species))
    {
    print(species[i])
    
    d=data_RA[data_RA$Species==species[i],]
    d=Biome_class(d)
    print(nrow(d))
    freq=c(nrow(d[d$biome_id==1,]),nrow(d[d$biome_id==2,]),
           nrow(d[d$biome_id==3,]), nrow(d[d$biome_id==4,]),
           nrow(d[d$biome_id==5,]), nrow(d[d$biome_id==6,]),
           nrow(d[d$biome_id==7,]),nrow(d[d$biome_id==8,]),
           nrow(d[d$biome_id==9,]),nrow(d[d$biome_id==10,]))
    print(freq/sum(freq))
    #ans= multinomial.test(freq, p_null, MonteCarlo = TRUE)      
    ans=chisq.test(freq, p=p_null, B=1000000) 
    #print(ans)
    pvals=c(pvals, ans$p.value)
    }
p.adjust(pvals, 'fdr')

[1] "anteater"
[1] 192
 [1] 0.687500000 0.072916667 0.000000000 0.098958333 0.041666667 0.000000000
 [7] 0.005208333 0.026041667 0.052083333 0.015625000


Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] "badger"
[1] 466
 [1] 0.137339056 0.139484979 0.002145923 0.008583691 0.328326180 0.006437768
 [7] 0.070815451 0.098712446 0.203862661 0.004291845


Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] "ground squirrel"
[1] 463
 [1] 0.144708423 0.215982721 0.000000000 0.006479482 0.254859611 0.028077754
 [7] 0.056155508 0.183585313 0.095032397 0.015118790


...

[1] "skunk"
[1] 310
 [1] 0.209677419 0.116129032 0.000000000 0.029032258 0.251612903 0.000000000
 [7] 0.074193548 0.141935484 0.170967742 0.006451613


Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] "spider"
[1] 3816
 [1] 0.271488470 0.161949686 0.003144654 0.053983229 0.136268344 0.034591195
 [7] 0.073375262 0.076519916 0.102725367 0.085953878
[1] "wren"
[1] 1117
 [1] 0.198746643 0.132497762 0.007162041 0.034019696 0.210384960 0.013428827
 [7] 0.102059087 0.094897046 0.158460161 0.048343778


Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] 1.145531e-47 1.422032e-27 5.279184e-22 2.604715e-12 1.020193e-90
 [6] 5.219917e-09 6.482544e-17 4.840874e-07 6.514781e-15 5.811078e-10
[11] 3.271161e-17 4.683287e-22 2.164693e-73 8.633633e-09 2.324052e-36
[16] 2.306402e-09

In [17]:
p.adjust(pvals, 'fdr')<0.05

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE

In [8]:
data_TS=read.csv("TrickSter_data4.csv")
pvals=c()
species=unique(data_TS$TrickSter)
for (i in 1:length(species))
    {
    print(species[i])
    
    d=data_TS[data_TS$TrickSter==species[i],]
    d=Biome_class(d)
    print(nrow(d))
    freq=c(nrow(d[d$biome_id==1,]),nrow(d[d$biome_id==2,]),
           nrow(d[d$biome_id==3,]), nrow(d[d$biome_id==4,]),
           nrow(d[d$biome_id==5,]), nrow(d[d$biome_id==6,]),
           nrow(d[d$biome_id==7,]),nrow(d[d$biome_id==8,]),
           nrow(d[d$biome_id==9,]),nrow(d[d$biome_id==10,]))
    print(freq/sum(freq))
    #ans= multinomial.test(freq, p_null, MonteCarlo = TRUE)      
    ans=chisq.test(freq, p=p_null, B=1000000) # because of rare data, chi squre data seems problem use multi-nominal test
    print(ans)
    pvals=c(pvals, ans$p.value)
    }
p.adjust(pvals, 'fdr')

[1] "raven"
[1] 79
 [1] 0.05063291 0.01265823 0.00000000 0.01265823 0.05063291 0.29113924
 [7] 0.24050633 0.03797468 0.07594937 0.22784810


Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”



	Chi-squared test for given probabilities

data:  freq
X-squared = 261.82, df = 9, p-value < 2.2e-16

[1] "skunk"
[1] 12
 [1] 0.0000000 0.0000000 0.0000000 0.0000000 0.1666667 0.0000000 0.1666667
 [8] 0.0000000 0.3333333 0.3333333


Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”



	Chi-squared test for given probabilities

data:  freq
X-squared = 31.541, df = 9, p-value = 0.000239

[1] "mink"
[1] 9
 [1] 0.0000000 0.0000000 0.0000000 0.0000000 0.2222222 0.0000000 0.0000000
 [8] 0.0000000 0.4444444 0.3333333


...

Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”



	Chi-squared test for given probabilities

data:  freq
X-squared = 9.1911, df = 9, p-value = 0.4198

[1] "wren"
[1] 9
 [1] 0.0000000 0.0000000 0.0000000 0.0000000 0.1111111 0.0000000 0.1111111
 [8] 0.0000000 0.3333333 0.4444444


Warning message in chisq.test(freq, p = p_null, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”



	Chi-squared test for given probabilities

data:  freq
X-squared = 36.896, df = 9, p-value = 2.746e-05



[1] 5.073959e-50 6.372739e-04 2.094997e-03 1.458166e-02 2.609555e-01
 [6] 5.359573e-01 3.717276e-12 2.932308e-10 1.341283e-06 4.478126e-01
[11] 1.945713e-02 1.573989e-02 3.261171e-03 2.637232e-03 4.478126e-01
[16] 8.788021e-05

In [9]:
p.adjust(pvals, 'fdr')<0.05

[1]  TRUE  TRUE  TRUE  TRUE FALSE FALSE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE
[13]  TRUE  TRUE FALSE  TRUE

In [10]:
# Trcikster vs real animals
data_RA=read.csv("Real_Biome_distributions2.csv")
data_TS=read.csv("TrickSter_data4.csv")
vals=c()
for (i in 1:length(species)){
    target=species[i]
    print(target)
    d_RA=data_RA[data_RA$Species==target,]
    d_TS=data_TS[data_TS$TrickSter==target, ]
    
    
    d_RA=Biome_class(d_RA)
    freq_RA=c(nrow(d_RA[d_RA$biome_id==1,]),nrow(d_RA[d_RA$biome_id==2,]),
           nrow(d_RA[d_RA$biome_id==3,]), nrow(d_RA[d_RA$biome_id==4,]),
           nrow(d_RA[d_RA$biome_id==5,]), nrow(d_RA[d_RA$biome_id==6,]),
           nrow(d_RA[d_RA$biome_id==7,]),nrow(d_RA[d_RA$biome_id==8,]),
           nrow(d_RA[d_RA$biome_id==9,]),nrow(d_RA[d_RA$biome_id==10,]))
    freq_RA=freq_RA/sum(freq_RA)
    
    d_TS=Biome_class(d_TS)
    freq_TS=c(nrow(d_TS[d_TS$biome_id==1,]),nrow(d_TS[d_TS$biome_id==2,]),
           nrow(d_TS[d_TS$biome_id==3,]), nrow(d_TS[d_TS$biome_id==4,]),
           nrow(d_TS[d_TS$biome_id==5,]), nrow(d_TS[d_TS$biome_id==6,]),
           nrow(d_TS[d_TS$biome_id==7,]),nrow(d_TS[d_TS$biome_id==8,]),
           nrow(d_TS[d_TS$biome_id==9,]),nrow(d_TS[d_TS$biome_id==10,]))
    freq_TS=freq_TS/sum(freq_TS)

    
    drop=c()
    for (i in 1:10){
       if (freq_TS[i]==0 && freq_RA[i]==0){
           drop=c(drop, i)
       }
    }


    if(length(drop)>0){
        ans=chisq.test(freq_TS[-c(drop)], p=freq_RA[-c(drop)], B=1000000)
        } else{
    ans=chisq.test(freq_TS, p=freq_RA, B=1000000)
    }  
    print(ans$p.value)
    vals=c(vals, ans$p.value)
}
#print(pvals)
p.adjust(vals, 'fdr')

[1] "raven"


Warning message in chisq.test(freq_TS, p = freq_RA, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] 0.9972327
[1] "skunk"


Warning message in chisq.test(freq_TS[-c(drop)], p = freq_RA[-c(drop)], B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] 0.01523418
[1] "mink"


...

Warning message in chisq.test(freq_TS, p = freq_RA, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] 0.9467336
[1] "wren"


Warning message in chisq.test(freq_TS, p = freq_RA, B = 1e+06):
“ カイ自乗近似は不正確かもしれません ”


[1] 0.9135839


[1] 0.9999968 0.2437469 0.9999968 0.9999968 0.9999968 0.9999968 0.9999968
 [8] 0.9999968 0.9999968 0.9999968 0.9999968 0.9999968 0.9999968 0.9999968
[15] 0.9999968 0.9999968